In [ ]:
import warnings

from dotenv import load_dotenv

# Suppress warning from google-genai when accessing text on function calls
warnings.filterwarnings("ignore", message="there are non-text parts in the response")

load_dotenv()

In [ ]:
from pathlib import Path

from schemas import JsonResume

type FileType = str | Path | None

DEFAULT_FILE = Path("../data/json_resume_example.json")


def get_json_resume() -> JsonResume:
    """Load a JSON resume from a file and validate it against the JsonResume schema.

    Returns:
        JsonResume: The validated JSON resume object.
    """
    with Path(DEFAULT_FILE).open(encoding="utf-8") as json_resume_path:
        json_resume = JsonResume.model_validate_json(json_resume_path.read())

    return json_resume

In [ ]:
from rich import print as rprint

json_resume = get_json_resume()

rprint(json_resume)

In [ ]:
import typing as t


def iterate_bfs(tree: dict) -> t.Generator[tuple[str, dict[str, t.Any]]]:
    """Iterate through a nested dictionary using BFS (Breadth-First Search).

    Args:
        tree (dict): The nested dictionary to traverse.

    Yields:
        tuple[str, dict]: A tuple containing the key and dictionary node for
            each dictionary found in the tree structure.
    """
    from collections import deque  # noqa: PLC0415

    queue: deque[tuple[str, dict | list]] = deque([("root", tree)])

    while queue:
        key, current_node = queue.pop()

        # Skip non-dict and non-list nodes
        if isinstance(current_node, dict):
            yield key, current_node

        # If the current node is a dictionary, we add its values to the queue
        #  (they are the child nodes)
        if isinstance(current_node, dict):
            for child_key, child_value in current_node.items():
                if isinstance(child_value, (dict, list)):
                    queue.appendleft((child_key, child_value))

        # The node can also be a list. We need to handle that case too.
        if isinstance(current_node, list):
            for item in current_node:
                if isinstance(item, (dict, list)):
                    queue.appendleft((key, item))

In [ ]:
import typing as t


def get_stacks_by_experience() -> dict[str, list[str]]:
    """Get technology stacks organized by experience type from the JSON resume.

    This function traverses the JSON resume structure using BFS (Breadth-First Search)
    to find all "keywords" fields and organizes them by their parent context
    (e.g., work experience, skills, projects, etc.).

    Returns:
        dict[str, list[str]]: A dictionary where keys represent the experience
            type or context (e.g., 'work', 'skills', 'projects') and values
            are sorted lists of unique technology stacks/keywords found in
            that context.
    """
    from collections import defaultdict  # noqa: PLC0415

    json_resume = get_json_resume()

    # Convert the Pydantic model to a dictionary
    dict_resume = json_resume.model_dump()

    # Iterate through the JSON structure using BFS
    stacks: defaultdict[str, list[str]] = defaultdict(list)
    for name, experience_node in iterate_bfs(dict_resume):
        if keywords := experience_node.get("keywords"):
            # Sometimes keywords can be None. We skip those cases.
            if keywords is None:
                continue
            stacks[name].extend(keywords)

    # Remove duplicates and sort the stacks
    stacks_sorted = {key: sorted(set(value)) for key, value in stacks.items()}

    return stacks_sorted

In [ ]:
def get_all_stacks() -> list[str]:
    """Get a sorted list of all unique technology stacks/keywords from the JSON resume.

    Returns:
        dict: A dictionary containing the status and a semicolon-separated string
            of all unique technology stacks/keywords.
    """
    stacks_by_experience = get_stacks_by_experience()
    all_stacks = set()
    for stacks in stacks_by_experience.values():
        all_stacks.update(stacks)
    return sorted(all_stacks)

## Embeddings

We seek to obtain some metric of similarity between experiences and skill stacks. To do this, we will use embeddings generated by a language model.

In [ ]:
from IPython.display import Markdown
from IPython.display import display as ip_display
from pydantic import BaseModel


class Formatable(t.Protocol):
    def format(self) -> str: ...


class Identifiable(t.Protocol):
    def get_id(self) -> str: ...


class WithType(t.Protocol):
    @property
    def item_type(self) -> str: ...


class Dumpable(t.Protocol):
    def model_dump(self) -> dict[str, t.Any]: ...


class Persistable(Formatable, Identifiable, WithType, Dumpable, t.Protocol):
    """Protocol for objects that can be formatted and identified for persistence."""

    pass


def iterate_over_formatables(
    json_resume: JsonResume,
) -> t.Iterator[Persistable]:
    if works := json_resume.work:
        yield from works

    if volunteers := json_resume.volunteer:
        yield from volunteers

    if certificates := json_resume.certificates:
        yield from certificates

    if projects := json_resume.projects:
        yield from projects

    if skills := json_resume.skills:
        yield from skills

    if interests := json_resume.interests:
        yield from interests


experencies: list[Persistable] = []
for exp in iterate_over_formatables(json_resume):
    experencies.append(exp)
    ip_display(Markdown(exp.format()))

In [ ]:
def find_experience_from_id(
    experience_id: str,
) -> Persistable | None:
    """Find an experience by its ID in the JSON resume.

    Args:
        experience_id (str): The ID of the experience to find.
    Returns:
        Persistable | None: The experience object if found, otherwise None.
    """
    json_resume = get_json_resume()
    for exp in iterate_over_formatables(json_resume):
        if exp.get_id() == experience_id:
            return exp
    return None


find_experience_from_id("interest.culture")

In [ ]:
from google import genai

client = genai.Client()

for m in client.models.list():
    supp_actions = m.supported_actions
    if supp_actions and "embedContent" in supp_actions:
        print(m.name)  # noqa: T201

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry
from google.genai import types
from google.genai.errors import APIError


# Define a helper to retry when per-minute quota is reached.
def is_retriable(e):
    return isinstance(e, APIError) and e.code in {429, 503}


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    EMBEDDING_MODEL = "models/text-embedding-004"
    OUTPUT_DIM = 768

    def __init__(
        self,
        document_mode: bool = True,
        embedding_model: str | None = None,
        output_dim: int | None = None,
    ):
        self.document_mode = document_mode
        self.embedding_model = embedding_model
        self.output_dim = output_dim

    @property
    def task_type(self) -> str:
        return "RETRIEVAL_DOCUMENT" if self.document_mode else "RETRIEVAL_QUERY"

    @retry.Retry(predicate=is_retriable)
    @t.override
    def __call__(self, input: Documents) -> Embeddings:
        response = client.models.embed_content(
            model=self.embedding_model or self.EMBEDDING_MODEL,
            contents=input,  # type: ignore
            config=types.EmbedContentConfig(
                task_type=self.task_type,
                output_dimensionality=self.output_dim or self.OUTPUT_DIM,
            ),
        )
        return [e.values for e in response.embeddings]  # type: ignore

In [ ]:
import chromadb

DB_NAME = "cv_embeddings_v2"

embed_fn = GeminiEmbeddingFunction(document_mode=True)

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)


db.add(
    documents=[e.format() for e in experencies],
    ids=[e.get_id() for e in experencies],
    metadatas=[{"item_type": e.item_type} for e in experencies],
)

In [ ]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = ", ".join([
    "Python",
    "SQL",
    "Machine Learning",
    "Data Mining",
    "Computer Vision",
    "GCP",
    "AWS",
    "Azure",
    "CI/CD pipelines",
    "Docker",
    "OpenCV",
    "PyTorch",
    "TensorFlow",
    "YOLO",
    "Detectron2",
])
query = ", ".join([
    "Python",
    "SQL",
    "Machine Learning",
])

result = db.query(query_texts=[query], n_results=10)
[all_passages] = result["documents"]  # type: ignore

Markdown(all_passages[0])

In [ ]:
from chromadb.base_types import Metadata


class ExperienceRetrieval(t.TypedDict):
    exp_id: str
    exp_type: str
    description: str
    distance: float


class ExperienceRetrievalResult(t.TypedDict, total=False):
    status: str
    retrieved_experiences: list[ExperienceRetrieval]
    message: str

In [ ]:
import json


def retrive_experiences_by_query(
    query: str,
    n_results: int,
) -> ExperienceRetrievalResult:
    """Retrieve experiences from the ChromaDB based on a query string.
    The embedding function will be switched to query mode during this operation.

    Args:
        query (str): The query string to search for.
        n_results (int): The number of top results to retrieve. If the value
            is less than or equal to zero, all experiences will be retrieved.

    Returns:
        dict[str, str]: A dictionary containing the status and retrieved experiences.

    Example:
        >>> retrive_experiences_by_query(
        ...     query="Python, SQL, Machine Learning",
        ...     n_results=2,
        ... )
        {
            "status": "success",
            "retrieved_experiences": "[\n  {\n    \"exp_id\": \"work_1\",\n    \"description\": \"Developed data pipelines using Python and SQL...\",\n    \"exp_type\": \"work\",\n    \"distance\": 0.12345\n  },\n  {\n    \"exp_id\": \"project_3\",\n    \"description\": \"Implemented machine learning models using Python...\",\n    \"exp_type\": \"project\",\n    \"distance\": 0.23456\n  }\n]"
        }
    """
    # Switch to query mode when generating embeddings.
    embed_fn.document_mode = False

    # Search the Chroma DB using the specified query.
    if n_results <= 0:
        n_results = db.count()

    try:
        result = db.query(query_texts=[query], n_results=n_results)

        documents: list[str] = []
        ids: list[str] = []
        metadatas: list[Metadata] = []
        distances: list[float] = []

        if result["documents"]:
            documents = result["documents"][0]

        if result["ids"]:
            ids = result["ids"][0]

        if result["metadatas"]:
            metadatas = result["metadatas"][0]

        if result["distances"]:
            distances = result["distances"][0]

        retrived_experiences = [
            ExperienceRetrieval(
                exp_type=str(metadata["item_type"]),
                description=passage,
                exp_id=exp_id,
                distance=distance,
            )
            for exp_id, passage, metadata, distance in zip(
                ids, documents, metadatas, distances, strict=False
            )
        ]

        return {"status": "success", "retrieved_experiences": retrived_experiences}
    except Exception as e:
        return {
            "status": "error",
            "message": str(e),
        }


retrived_exps = retrive_experiences_by_query(
    query=(
        "Python, SQL, Machine Learning, Data Mining, Computer Vision, GCP, AWS,"
        " Azure, CI/CD pipelines, Docker, OpenCV, PyTorch, TensorFlow, YOLO,"
        " Detectron2"
    ),
    n_results=0,
)

# CV Enhancer

El objetivo es crear un agente que ayude a mejorar un currículum vitae (CV) agregando y enfocando las experiencias laborales y habilidades relevantes para un puesto de trabajo específico.

In [ ]:
from dataclasses import dataclass

from google.adk.models.google_llm import Gemini

retry_config = types.HttpRetryOptions(
    attempts=10,  # Maximum retry attempts
    initial_delay=1,
    max_delay=60,
    exp_base=3,  # Delay multiplier
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)


@dataclass
class CvSaverConfig:
    job_offer_summarizer_model: Gemini
    experience_enhancement_model: Gemini
    critic_model: Gemini
    cv_enhancement_model: Gemini


config = CvSaverConfig(
    job_offer_summarizer_model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config,
    ),
    experience_enhancement_model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config,
    ),
    critic_model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config,
    ),
    cv_enhancement_model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config,
    ),
)

rprint(config)

In [ ]:
from google.adk.agents import Agent, BaseAgent
from google.adk.agents.invocation_context import InvocationContext
from google.adk.events import Event


class JobOfferSaverAgent(BaseAgent):
    """This agent saves the job offer details into the session state."""

    @t.override
    async def _run_async_impl(self, ctx: InvocationContext) -> t.AsyncGenerator[Event]:
        # Get the first event from the session
        first_event = ctx.session.events[0]

        # If the first event is not from the user, we ignore it
        if first_event.author != "user":
            return

        # Otherwise, we try to extract the job offer from the first event
        job_offer = None
        if (content := first_event.content) and (parts := content.parts):
            part = parts[0]
            if job_offer := part.text:
                # Save the job offer in the session state
                ctx.session.state["job_offer"] = job_offer

        # Finally, we yield the first event back to the session
        yield Event(author=self.name)


job_offer_saver_agent = JobOfferSaverAgent(name="JobOfferSaverAgent")

## Job Summarizer

Since the first input is a job offer, a model is needed to summarize the job offer in order to extract the key points. This will help to avoid spending too many tokens when delegating the task of improving the CV.

In [ ]:
from pydantic import Field


class JobOfferSummarized(BaseModel):
    job_description: str = Field(
        ...,
        description="Concise description of the job",
    )
    requirements: list[str] = Field(
        ...,
        description="Key requirements listed clearly",
    )
    tech_stack: list[str] = Field(
        ...,
        description="Technologies and tools mentioned",
    )

    __EXAMPLE__ = {
        "job_description": "Develop and maintain web applications.",
        "requirements": ["Python", "Django", "REST APIs"],
        "tech_stack": ["AWS", "Docker", "PostgreSQL"],
    }


assert JobOfferSummarized(  # noqa: S101
    **JobOfferSummarized.__EXAMPLE__
), "Example does not conform to schema"

job_offer_summ = JobOfferSummarized(**JobOfferSummarized.__EXAMPLE__)

job_offer_summ

In [ ]:
from google.adk.models.google_llm import Gemini
from google.genai import types

PROMPT = f"""
You are a Job Offer Summarizer. Your only task is to read the provided job offer
and extract the key information such as the description, requirements, stack
and provide a concise summary. Do not add any additional information
(such as benefits, the title, the company...) or opinions. Try to summarize
in 300 words maximum.

The output must be only the summarized job offer without any additional
commentary.

Try to be as specific as possible when extracting the tech stack.

Here is an example of the output format:
---
{json.dumps(JobOfferSummarized.__EXAMPLE__, indent=2)}
---

You MUST RETURN the output in the EXACT FORMAT as shown above, without any additional text.
"""

job_offer_summarizer_agent = Agent(
    name="JobOfferSummarizerAgent",
    model=config.job_offer_summarizer_model,
    description=(
        "An agent that summarizes job offers by extracting key information and"
        " presenting it clearly to assist job seekers in understanding"
        " the opportunities."
    ),
    instruction=PROMPT,
    output_key="summarized_job_offer",
    output_schema=JobOfferSummarized,
)

print("✅ Job Offer Summarizer Agent defined.")  # noqa: T201

## Experience Enhancement Agent

In [ ]:
retrived_exps["retrieved_experiences"][0]

In [ ]:
EXAMPLES = {
    "work": {
        "summary": "Description…",
        "highlights": ["Started the company"],
        "keywords": ["leadership", "entrepreneurship"],
    },
    "volunteer": {
        "summary": "Description…",
        "highlights": ["Volunteered at local shelter"],
        "keywords": ["community", "helping others"],
    },
    "certificates": {
        "summary": "Description…",
        "keywords": ["certification", "achievement"],
    },
    "projects": {
        "summary": "Description…",
        "highlights": ["Developed a web app"],
        "keywords": ["Python", "Django", "AWS"],
    },
    "skills": {
        "keywords": ["Python", "Machine Learning", "Data Analysis"],
    },
    "interests": {
        "keywords": ["hiking", "photography", "travel"],
    },
}


def get_output_example(exp_kind: str) -> dict[str, str]:
    """Get an example output for a given experience kind.

    Args:
        exp_kind (str): A type of experience (e.g., 'work', 'project', 'certificate').

    Returns:
        dict[str, str]: A dictionary containing the status and either an example output or an error message.
    """
    if exp_kind not in EXAMPLES:
        return {
            "status": "error",
            "message": (
                f"Unknown experience kind: {exp_kind}."
                f" Plese use one of {list(EXAMPLES.keys())}"
            ),
        }

    return {
        "status": "success",
        "example": json.dumps(EXAMPLES[exp_kind], indent=2),
    }

In [ ]:
PROMPT_INITIAL_EXP_ENHANCEMENT = """
You are an experience enhancer. Your task is to improve the provided
description of experience to better align it with the job posting summary.

The main goal is to ensure that the experience description highlights
the skills, technologies, and accomplishments that are most relevant to
the job requirements and technology stack mentioned in the job posting.

Use the tool `get_output_example(exp_kind: str)` to get an example output
for the kind of experience you are enhancing.

YOU MUST RETURN the output in the EXACT FORMAT as shown in the example,
without any additional text. NOT FOLLOWING THE FORMAT WILL CAUSE ERRORS.

Job Posting Summary:
{{summarized_job_offer}}

Experience Description:
{{experience_{agent_id}}}

OUTPUT RULES:
- The summary/description must be concise and focused on relevant skills
    and accomplishments. Avoid unnecessary details. Try to keep it
    in 1-2 sentences.
- The highlights must be specific achievements or contributions,
    quantifiable where possible. Try to include 2-3 highlights. Each highlight
    should start with a strong action verb. Try to keep it in a sentence each.
- The keywords must include relevant technologies, tools, and skills.
    Try to include 3-5 keywords.
- Do not invent details.. Only use the information provided in the
    experience description.
"""

PROMPT_CRITIC = """
You are a constructive critic. Your task is to review the provided
draft of experience and provide feedback on how well it aligns with the
job posting summary. Identify areas of improvement, suggest enhancements,
and highlight any discrepancies between the draft and the job requirements.

Job Posting Summary:
{{summarized_job_offer}}

Experience Draft:
{{enhanced_experience_{agent_id}}}

- If the draft is well-aligned, you MUST respond with the exact phrase: "APPROVED"
- Otherwise, provide 1-3 specific, actionable suggestions for improvement,
    of the draft to better align it with the job posting summary.
"""

PROMPT_REFINER_EXP_ENHANCEMENT = """
You are an experience enhancer. Your task is to improve the provided
description of experience to better align it with the job posting summary.
The main goal is to ensure that the experience description highlights
the skills, technologies, and accomplishments that are most relevant to
the job requirements and technology stack mentioned in the job posting.

Use the tool `get_output_example(exp_kind: str)` to get an example output
for the kind of experience you are enhancing.

YOU MUST RETURN the output in the EXACT FORMAT as shown in the example,
without any additional text. NOT FOLLOWING THE FORMAT WILL CAUSE ERRORS.

Job Posting Summary:
{{summarized_job_offer}}

Original Experience Description:
{{experience_{agent_id}}}

Current Experience Draft:
{{enhanced_experience_{agent_id}}}

Critique Feedback:
{{critique_{agent_id}}}

- IF THE CRITIQUE IS "APPROVED", you MUST call the `exit_loop` function and nothing else.
- OTHERWISE, use the critique feedback to make specific improvements
    to the experience draft to better align it with the job posting summary.

OUTPUT RULES:
- The summary/description must be concise and focused on relevant skills
    and accomplishments. Avoid unnecessary details. Try to keep it
    in 1-2 sentences.
- The highlights must be specific achievements or contributions,
    quantifiable where possible. Try to include 2-3 highlights. Each highlight
    should start with a strong action verb. Try to keep it in a sentence each.
- The keywords must include relevant technologies, tools, and skills.
    Try to include 3-5 keywords.
- Do not invent details.. Only use the information provided in the
    experience description.
"""

In [ ]:
def exit_loop() -> dict[str, t.Any]:
    """
    Call this function ONLY when the critique is 'APPROVED',
    indicating the experience is finished and no more changes are needed."""
    return {
        "status": "approved",
        "message": "Experience approved. Exiting refinement loop.",
    }

In [ ]:
import uuid

from google.adk.agents import LoopAgent, SequentialAgent
from google.adk.tools import FunctionTool


def create_experience_enhancement_agent(
    agent_id: str | None = None, max_iterations: int = 1
) -> SequentialAgent:
    # Use a unique agent ID if not provided
    if agent_id is None:
        agent_id = str(uuid.uuid4())[:8]

    initial_experience_enhancement_agent = Agent(
        name="InitialExperienceEnhancementAgent" + agent_id,
        model=config.experience_enhancement_model,
        description=(
            "An agent that enhances a first draft of experience"
            " descriptions to better align them with job requirements and"
            " technology stack."
        ),
        instruction=PROMPT_INITIAL_EXP_ENHANCEMENT.format(agent_id=agent_id),
        output_key="enhanced_experience_" + agent_id,
        tools=[FunctionTool(get_output_example)],
    )

    critic_agent = Agent(
        name="CriticAgent" + agent_id,
        model=config.critic_model,
        description=(
            "An agent that critiques and provides feedback on a draft of experience."
        ),
        instruction=PROMPT_CRITIC.format(agent_id=agent_id),
        output_key="critique_" + agent_id,
    )

    refiner_experience_enhancement_agent = Agent(
        name="RefinerExperienceEnhancementAgent" + agent_id,
        model=config.experience_enhancement_model,
        description=(
            "An agent that enhances a draft of experience descriptions based on"
            " feedback from a critic agent to better align them with job"
            " requirements and technology stack."
        ),
        instruction=PROMPT_REFINER_EXP_ENHANCEMENT.format(agent_id=agent_id),
        output_key="enhanced_experience_" + agent_id,
        tools=[FunctionTool(exit_loop), FunctionTool(get_output_example)],
    )

    experience_refinement_loop = LoopAgent(
        name="ExperienceRefinementLoopAgent" + agent_id,
        sub_agents=[
            critic_agent,
            refiner_experience_enhancement_agent,
        ],
        max_iterations=max_iterations,
    )

    experience_enhancement_sequence = SequentialAgent(
        name="ExperienceEnhancementAgent" + agent_id,
        sub_agents=[
            initial_experience_enhancement_agent,
            experience_refinement_loop,
        ],
    )

    return experience_enhancement_sequence

In [ ]:
class ExperienceLimits(t.TypedDict, total=False):
    work: int
    volunteer: int
    certificates: int
    projects: int
    skills: int
    interests: int


ExperienceLimitsKeys = t.Literal[
    "work",
    "volunteer",
    "certificates",
    "projects",
    "skills",
    "interests",
]

In [ ]:
retrived_exps = retrive_experiences_by_query(
    query="Python, SQL, Machine Learning",
    n_results=0,
)["retrieved_experiences"]


def limit_experiences_by_type(
    experiences: list[ExperienceRetrieval],
    limits: ExperienceLimits,
) -> list[ExperienceRetrieval]:
    """Limit the number of experiences by type.

    Args:
        experiences (list[ExperienceRetrieval]): The list of experiences to limit.
        limits (ExperienceLimits): The limits for each experience type.

    Returns:
        list[ExperienceRetrieval]: The limited list of experiences.
    """
    from collections import Counter  # noqa: PLC0415

    counts: Counter[str] = Counter()
    limited_experiences: list[ExperienceRetrieval] = []

    for exp in experiences:
        exp_type: ExperienceLimitsKeys = exp["exp_type"]  # type: ignore
        if exp_type in limits and counts[exp_type] < limits[exp_type]:
            limited_experiences.append(exp)
            counts.update([exp_type])

    return limited_experiences


limit_experiences_by_type(
    retrived_exps,
    limits={
        "work": 2,
    },
)

In [ ]:
import logging

from google.adk.agents import ParallelAgent


class CVEnhancementAgent(BaseAgent):
    """This agent enhances a CV based on the job offer summary and
    retrieved experiences.
    """

    experience_limits: ExperienceLimits | None = None
    max_refinement_iterations: int = 1

    @t.override
    async def _run_async_impl(self, ctx: InvocationContext) -> t.AsyncGenerator[Event]:
        logging.info(f"[{self.name}] Starting CV Enhancement Agent.")

        # Get the job offer summary from the session state
        job_offer_summary = ctx.session.state.get("summarized_job_offer", None)
        if job_offer_summary is None:
            logging.error(f"[{self.name}] No job offer summary found in session state.")
            return

        # Retrieve the experiences using the retrive_experiences_by_query tool
        #  from the session state
        retrived_exps = retrive_experiences_by_query(
            query=", ".join(job_offer_summary["tech_stack"]),
            n_results=0,
        )["retrieved_experiences"]

        # Limit the number of experiences if limits are provided
        if self.experience_limits:
            retrived_exps = limit_experiences_by_type(
                retrived_exps,
                limits=self.experience_limits,
            )
        logging.info(
            f"[{self.name}] Retrieved {len(retrived_exps)} experiences for enhancement."
        )
        logging.debug(f"[{self.name}] Experiences: {retrived_exps}")

        # Tag each experience with a unique ID.
        #  Also save in the state the experience descriptions
        tagged_experiences: dict[str, ExperienceRetrieval] = {}
        for exp in retrived_exps:
            tag_id = "_" + exp["exp_id"].replace(".", "_")
            tagged_experiences[tag_id] = exp
            ctx.session.state[f"experience_{tag_id}"] = exp["description"]

        # Create a list of experience enhancement agents for parallel execution
        enhancement_agents = [
            create_experience_enhancement_agent(
                agent_id=agent_id, max_iterations=self.max_refinement_iterations
            )
            for agent_id in tagged_experiences
        ]

        parallel_enhancement_agent = ParallelAgent(
            name="ParallelExperienceEnhancementAgent",
            sub_agents=enhancement_agents,
        )

        # Run the parallel enhancement agent
        async for event in parallel_enhancement_agent.run_async(ctx):
            yield event

        # Parse the enhanced experience from the session state
        parsed_enhanced_experiences: dict[str, dict[str, t.Any]] = {}
        for agent_id in tagged_experiences:
            enhanced_experience: str | None = ctx.session.state.get(
                f"enhanced_experience_{agent_id}"
            )
            if enhanced_experience:
                enhanced_exp = enhanced_experience.replace("```json", "").replace(
                    "```", ""
                )
                parsed_enhanced_experiences[agent_id] = json.loads(enhanced_exp)

        logging.info(f"[{self.name}] Parsed enhanced experiences.")
        logging.debug(
            f"[{self.name}] Enhanced Experiences: {parsed_enhanced_experiences}"
        )

        # Build a json resume with the enhanced experiences
        json_resume_original = get_json_resume()
        json_resume_enhanced: dict[str, dict | list] = {}
        if basics := json_resume_original.basics:
            json_resume_enhanced["basics"] = basics.model_dump()

        for agent_id, enhanced_exp in parsed_enhanced_experiences.items():
            exp_id = tagged_experiences[agent_id]["exp_id"]
            exp_type = tagged_experiences[agent_id]["exp_type"]
            if exp_type not in json_resume_enhanced:
                json_resume_enhanced[exp_type] = []
            experience_obj = find_experience_from_id(exp_id).model_dump()  # type: ignore
            if experience_obj:
                for field, value in enhanced_exp.items():
                    experience_obj[field] = value
                json_resume_enhanced[exp_type].append(experience_obj)  # type: ignore

        logging.info(f"[{self.name}] CV Enhancement Agent completed.")
        yield Event(
            author=self.name,
            content={
                "parts": [{
                    "text": JsonResume(**json_resume_enhanced).model_dump_json(indent=2)
                }]
            },
        )


cv_enhancement_agent = CVEnhancementAgent(
    name="CVEnhancementAgent",
    experience_limits=ExperienceLimits(
        work=3,
        projects=1,
        certificates=1,
        skills=1,
    ),
)

In [ ]:
from google.adk.agents import SequentialAgent
from google.adk.runners import InMemoryRunner

root_agent = SequentialAgent(
    name="CvEnhancerRootAgent",
    sub_agents=[
        job_offer_saver_agent,
        job_offer_summarizer_agent,
        cv_enhancement_agent,
    ],
)

In [ ]:
from google.adk.plugins.logging_plugin import LoggingPlugin

runner = InMemoryRunner(
    agent=root_agent,
    plugins=[
        LoggingPlugin(),
    ],
)

print("✅ Runner created.")  # noqa: T201

In [ ]:
# Offer obtained from: https://www.linkedin.com/jobs/collections/?currentJobId=4324487265

response = await runner.run_debug("""
We are still looking for talent… and we would love for you to join our team!

For over 25 years, UST has worked alongside the world's best companies to make a real impact through business transformation. Driven by technology, inspired by people, and guided by our purpose, UST supports clients from design to implementation. Together, with more than 30,000 employees in 30 countries, we build to create limitless impact, reaching billions of lives in the process.


We are looking for an AI Engineer, to join a strategic project supporting data platform modernization.



UST is looking for a candidate with strong Python expertise, proven experience building applications using LLMs, and hands-on exposure to agentic AI frameworks.



What We're Looking For:

Experience: 6 to 8 years of professional experience.
Language: Advanced English B2 - C1
Strong programming expertise in Python.
Proven experience building applications using any LLMs.
1 to 2 years of hands-on experience with agentic AI frameworks.
Familiarity with cloud platforms.


Why Join Us:

Work in a remote and global environment
Be part of a mission-critical application team
Work in a supportive, collaborative environment
Opportunity to mentor others and influence product direction
Gain exposure to cloud migration and modern development practices


UST is waiting for you!
""")

In [ ]:
last_response = response[-1]

enhanced_cv = None
if (content := last_response.content) and (parts := content.parts):
    part = parts[0]
    if cv_json := part.text:
        enhanced_cv = JsonResume.model_validate_json(cv_json)

In [ ]:
rprint(enhanced_cv)